In [1]:
# Just ignore this shit
import sys
!{sys.executable} -m pip install openpyxl

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

# starts here
hr_data = pd.read_excel("data-2.xlsx", sheet_name='HR Data', engine='openpyxl')
exit_data = pd.read_excel("data-2.xlsx", sheet_name='HR Exit', engine='openpyxl')
    
# Just printing the data, to visualize better
print ("HR Data length:", len(hr_data))
print ("Exist Data length:", len(exit_data))

HR Data length: 10098
Exist Data length: 1838


In [2]:
# How many voluntary terminated employees
voluntary_exit_data = exit_data.loc[exit_data['Name of action type'] == 'Voluntary Termination']
print("Voluntary terminations: ", len(voluntary_exit_data))

Voluntary terminations:  996


In [3]:
# Let's make a new column and fill it in, 
# For exit data, if Voluntary Terminal == 1, else 0
exit_data['Voluntary Turnover'] = np.where(exit_data['Name of action type'] == 'Voluntary Termination', 1, 0)

# For Hr data, If the employee, is found in the HR exit list of voluntary terminated employees, 
# we update to 1, it otherwise ... 0 . that's what i'm going with 
matching_volun_term_employees = voluntary_exit_data['EmployeeID'].values
hr_data['Voluntary Turnover'] = hr_data['EmployeeID'].isin(matching_volun_term_employees).astype(int)

# Just printing the data, to visualize
print (exit_data[['EmployeeID', 'Voluntary Turnover']])
print (hr_data[['EmployeeID', 'Voluntary Turnover']])

      EmployeeID  Voluntary Turnover
0           7765                   1
1           8644                   1
2           1751                   0
3           4394                   0
4           2597                   0
...          ...                 ...
1833        5492                   1
1834       10146                   1
1835       10057                   1
1836        3726                   1
1837        2676                   1

[1838 rows x 2 columns]
       EmployeeID  Voluntary Turnover
0            6256                   0
1            8849                   0
2           10342                   0
3            4731                   1
4           10317                   0
...           ...                 ...
10093        8831                   0
10094        9137                   0
10095        3898                   1
10096        1929                   0
10097        9576                   1

[10098 rows x 2 columns]


In [4]:
#For 'ALL turnover'. basically all data from exit_hr?
print ("All exit hr employees:", len(exit_data))

#Count number of inactive accounts? like you did in your answer, to take into consideration as well
print("All inactive accounts from hr data:", len(hr_data.loc[hr_data['Name'] == 'Inactive']))

All exit hr employees: 1838
All inactive accounts from hr data: 114


In [5]:
# So let's make a new column and fill it in,

# For exit data, they're all "1" since they are all terminated I guess?, so all 1s
exit_data['All Turnover'] = 1

# For hr data, we can go with the accounts that are inactive, like you did cuz i'm not sure at this point
hr_data['All Turnover'] = np.where(hr_data['Name'] == 'Inactive', 1, 0)

# Just printing the data, to visualize
print (exit_data[['EmployeeID', 'All Turnover']])

# printed only the ones that are have turnover
print (hr_data[['EmployeeID', 'All Turnover']].loc[hr_data['All Turnover'] == 1])

      EmployeeID  All Turnover
0           7765             1
1           8644             1
2           1751             1
3           4394             1
4           2597             1
...          ...           ...
1833        5492             1
1834       10146             1
1835       10057             1
1836        3726             1
1837        2676             1

[1838 rows x 2 columns]
       EmployeeID  All Turnover
70           7879             1
104          4076             1
119          2792             1
254         10247             1
281          7556             1
...           ...           ...
9311         3625             1
9485         3175             1
9798         1316             1
9978         9709             1
10003        1652             1

[114 rows x 2 columns]


In [6]:
# Tenure date ...
# So every employee, from the date they started till date of report (in days)? .. aight
 
# Here we go through every column, and parse the string into a datetime object that we can do some nice operations on
# BTW: here we use errors='coerce' to force the function 'to_datetime' to kinda ignore invalid dates ish
hr_data['Original Hire Date'] = pd.to_datetime(hr_data['Original Hire Date'], errors='coerce')
exit_data['OrgHire'] = pd.to_datetime(exit_data['OrgHire'], errors='coerce')

# So where the hell is the date of the report?, but aight ill just make one
hr_data['Report date'] = pd.to_datetime('2014-01-15')
exit_data['Report date'] = pd.to_datetime('2014-01-15')

# Now we calculate the dates
hr_data['Firm Tenure days'] = (hr_data['Report date'] - hr_data['Original Hire Date']).dt.days
exit_data['Firm Tenure days'] = (exit_data['Report date'] - exit_data['OrgHire']).dt.days

# Just printing the reuslts to visualize
# Looks like we have some negative days in HR_exit data, some people got hired after the date Jan 15 2015,
# I mean if we just have a more recent report date should be fine
print (hr_data[['EmployeeID', 'Firm Tenure days']])
print (exit_data[['EmployeeID', 'Firm Tenure days']])

       EmployeeID  Firm Tenure days
0            6256           15129.0
1            8849           14659.0
2           10342           14593.0
3            4731           13624.0
4           10317           13456.0
...           ...               ...
10093        8831             702.0
10094        9137             520.0
10095        3898             520.0
10096        1929             492.0
10097        9576             454.0

[10098 rows x 2 columns]
      EmployeeID  Firm Tenure days
0           7765             14191
1           8644             11916
2           1751              9872
3           4394              9579
4           2597              9508
...          ...               ...
1833        5492              7856
1834       10146               723
1835       10057               -75
1836        3726              -159
1837        2676              -222

[1838 rows x 2 columns]


In [7]:
# Job tenure date
# Very similar to above

# Again we go through every column, and parse the string into a datetime object that we can do some nice operations on
# BTW: we also include the error='coerce' option
hr_data['Current Hire Date'] = pd.to_datetime(hr_data['Current Hire Date'], errors='coerce')
exit_data['Cur.Hire'] = pd.to_datetime(exit_data['Cur.Hire'], errors='coerce')

# We already created the report date, so we don't need to re-create it ... 

# Now we calculate the dates
hr_data['Job Tenure days'] = (hr_data['Report date'] - hr_data['Current Hire Date']).dt.days
exit_data['Job Tenure days'] = (exit_data['Report date'] - exit_data['Cur.Hire']).dt.days

# And finally we print to visualize the results
# I guess results are similar, cuz those 2 fields are prety much always the same
print (hr_data[['EmployeeID', 'Job Tenure days']])
print (exit_data[['EmployeeID', 'Job Tenure days']])

       EmployeeID  Job Tenure days
0            6256            15129
1            8849            14659
2           10342            14593
3            4731            13624
4           10317            10166
...           ...              ...
10093        8831              702
10094        9137              520
10095        3898              520
10096        1929              492
10097        9576              454

[10098 rows x 2 columns]
      EmployeeID  Job Tenure days
0           7765            14191
1           8644            11916
2           1751             9872
3           4394             8411
4           2597             9508
...          ...              ...
1833        5492             7856
1834       10146              723
1835       10057              -75
1836        3726             -159
1837        2676             -222

[1838 rows x 2 columns]


In [8]:
# So I have no clue wtf is correlation between 2 columns, but we got something for that in the pandas library
# Guess i'll do it for both data sets
print ('Correlation:', hr_data['Firm Tenure days'].corr(hr_data['All Turnover']))
print ('Correlation:', exit_data['Firm Tenure days'].corr(hr_data['All Turnover']))

Correlation: 0.004528228834491616
Correlation: -0.031804855147292206


In [9]:
# Pivot table
print (pd.pivot_table(hr_data, values='Voluntary Turnover', index=['PS text'], aggfunc={np.mean}))

                     mean
PS text                  
Bus Development  0.040000
Business Dev     0.000000
CHC Marketing    0.150000
CHC Nat          0.000000
CHC Sales        0.000000
Development      0.077895
Dist Warehouse   0.064516
Drug Reg Affair  0.018868
Engineering      0.071584
Environ Affairs  0.000000
Finance          0.047244
Govt. Affairs    0.071429
HR               0.116279
IT               0.035354
Interns Only     1.000000
Legal            0.041096
Maintenance      0.111111
Management       0.103093
Materials Mgmt.  0.071429
Medical          0.067754
Mgmt Support     0.000000
NOBL             0.048387
PM Comm Ops      0.083333
PM Field Sales   0.087123
PM Marketing     0.095238
PM Mgmt & Admin  0.090909
PM Mngd Mkt Fld  0.080000
PM Mngd Mkt Off  0.043478
PM Sales Admin   0.000000
Production       0.058703
Public Affairs   0.150000
Purchasing       0.120879
QA - Quality     0.074675
R & D            0.111111
R&D              0.067692
Research         0.079487
Service Cent

In [10]:
# I guess that's it! I'm pretty sure not everything is correct, but should be close enough i guess!

In [15]:
# Download data frames for verifications
hr_data.to_csv('HR_data.csv', index = False)
exit_data.to_csv('Exit_data.csv', index = False)